#### Convert the original dataset files (extension .mat) to CSV format.
 - The dataset comprises 480 MAT files (MATLAB files).

#### Import libraries.

In [6]:
import scipy.io
import pandas as pd
import numpy as np

#### Function that returns the scale (stress level) assigned by a given individual to perform a given task. 

In [7]:

def get_scale(data_frame, file_name):
    # Returns the scale associated to a certain file (each task).
    # Parameters:
    #    data_frame: dataframe containing all scales.
    #    file_name (str): file name to be processed.
    # Return:
    #    scale (str): scale value.
    file_itens = file_name.split('_')
    task_desc = ""
    sub_number = ""
    try_number = ""
    if (file_itens[0] == "Mirror") :
        task_desc = file_itens[0]
        sub_number = file_itens[3]
        try_number = file_itens[4].split('.')[0]
    else :
        task_desc = file_itens[0]
        sub_number = file_itens[2]
        try_number = file_itens[3].split('.')[0]
    line_index = data_frame[data_frame['Subject'] == int(sub_number)].index[0]
    column_name = ""
    if (task_desc == "Arithmetic") :
        column_name = "Maths" + try_number[-1]
    elif (task_desc == "Mirror") :
        column_name = "Symmetry" + try_number[-1]
    elif (task_desc == "Stroop") :
        column_name = "Stroop" + try_number[-1]
    else :
        return 0
    return data_frame.iloc[line_index][column_name]


#### Function that converts the list of MAT files into CSV format.
- The result is grouped into a single .CSV file..

In [22]:

def join_files(files_list):
    # Converts the MAT files list to a single CSV file.
    # Parameters:
    #    file_list (list): list of files.
    print("Init join_files function")
    all_features = {}
    all_features[0] = np.empty(0, dtype = int)
    for index in range(1, 33):
        all_features[index] = np.empty(0)
    label_index = index + 1
    all_features[label_index] = np.empty(0, dtype = int)
    all_scales = pd.read_excel('../dataset/scales.xls')
    record_number = 1
    for file_name in files_list:
        file_content = scipy.io.loadmat('../dataset/matfiles/' + file_name) 
        for x in file_content:
            # Use only numpy.ndarray element (electroencephalogram measurements).
            if isinstance(file_content[x], np.ndarray):
                features_count = 1
                for y in file_content[x]:
                    # Variable y contains all measurements for each feature.
                    all_features[features_count] = np.append(all_features[features_count], y)
                    features_count += 1
                for y in range(0, len(file_content[x][0])):
                    label_value = get_scale(all_scales, file_name)
                    all_features[features_count] = np.append(all_features[features_count], label_value)
                    all_features[0] = np.append(all_features[0], record_number)
                    record_number += 1
    print("Start writing CSV file.")
    df = pd.DataFrame.from_dict(all_features)
    df.rename(columns = {0: 'recordNumber', 1: 'CZ', 2: 'FZ', 3: 'Fp1', 4: 'F7', 5: 'F3', 6: 'FC1', 7: 'C3',
                         8: 'FC5', 9: 'FT9', 10: 'T7', 11: 'CP5', 12: 'CP1', 13: 'P3', 14: 'P7', 15: 'PO9',
                         16: 'O1', 17: 'PZ', 18: 'OZ', 19: 'O2', 20: 'PO10', 21: 'P8', 22: 'P4', 23: 'CP2',
                         24: 'CP6', 25: 'T8', 26: 'FT10', 27: 'FC6', 28: 'C4', 29: 'FC2', 30: 'F4',
                         31: 'F8', 32: 'Fp2', 33: 'Scale'}, inplace = True)
    df.to_csv('../dataset/original-sam-dataset.csv', index = False,  sep='|')
    print("Finish writing CSV file.")


#### Generate the list with the names of the MAT files to be converted to CSV and call the conversion function.

In [23]:
files_list = []

tasks_list = ["Arithmetic", "Mirror_image", "Relax", "Stroop"]

for task in tasks_list:
    for subject in range(0, 40):
        for trials in range(0, 3):
            files_list.append(task + "_sub_" + str(subject + 1) + "_trial" + str(trials + 1) + ".mat")

join_files(files_list)

Init join_files function
Start writing CSV file.
Finish writing CSV file.
